In [8]:
#### SETTING CUDA SUPPORT FOR SPACY
!pip install -U 'spacy[cuda-autodetect,transformers,lookups]'

#### CHECKING CUDA SUPPORT AND VERSION
#import torch
#torch.cuda.is_available()
#!nvcc --version
#import cupy
#cupy.show_config()
#import spacy
#spacy.require_gpu()

##### INSTALLATION OF THE REST OF LIBRARIES FOR NLP
!pip install wordcloud
!pip install nltk
!pip install spacy-streamlit --pre
!python -m spacy download en_core_web_lg
#!python -m spacy init fill-config ./configs/base_config.cfg ./configs/config.cfg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 458.6/458.6 kB 39.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 26.3 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 95.0 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 kB 43.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 812.8/812.8 kB 84.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 237.5/237.5 kB 53.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.1/51.1 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 338.8/338.8 kB 60.3 MB/s eta 0:00:00
  Created wheel for validators: filename=validators-0.20.0-py3-none-any.whl size=19580 sha256=fed044a59ec010ba7c783e965e61ab16e74f957c6ed

In [9]:
#### INSTALLATION OF LIBRARIES FOR SPARK
!pip install findspark
!sudo apt update
!sudo apt install openjdk-17-jdk -y
#!curl -JLO 'https://apache.osuosl.org/spark/spark-3.3.1/spark-3.3.1-bin-hadoop3.tgz'
!tar xvf spark-3.3.1-bin-hadoop3.tgz
!mv spark-3.3.1-bin-hadoop3 /opt/spark
!pip install pyspark

Get:1 https://deb.nodesource.com/node_16.x focal InRelease [4583 B]
Get:2 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]      
Get:3 http://archive.ubuntu.com/ubuntu focal InRelease [265 kB]                
Get:4 https://deb.nodesource.com/node_16.x focal/main amd64 Packages [774 B]   
Get:5 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease [18.1 kB] 
Get:6 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal/main amd64 Packages [29.5 kB]
Get:7 http://security.ubuntu.com/ubuntu focal-security/restricted amd64 Packages [1712 kB]
Get:8 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]        
Get:9 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]     m
Get:10 http://archive.ubuntu.com/ubuntu focal/universe amd64 Packages [11.3 MB]
Get:11 http://security.ubuntu.com/ubuntu focal-security/universe amd64 Packages [965 kB]
Get:12 http://security.ubuntu.com/ubuntu focal-security/multiverse amd64 Packages [27.5 kB]
Get:13 http

In [10]:
#### DIRECTORY SETTING FOR SPARK 
import os
os.environ["JAVA_HOME"] = "/usr"
os.environ["SPARK_HOME"] = "/opt/spark"

In [11]:
##### IMPORTING LIBRARIES

#### WORDCLOUD
from wordcloud import WordCloud
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords = stopwords.words('english')

#### BASIC
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

#### SPARK
import findspark
findspark.init()
from pyspark.sql import SparkSession
import pyspark.pandas as ps
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.sparkContext.setLogLevel("OFF")

#### NLP IMPORTING FOR PIPELINE
from spacy import displacy
from spacy.util import minibatch, compounding
from typing import Set, List, Tuple
from spacy.tokens import DocBin
import spacy
nlp = spacy.load('en_core_web_lg')

#### CUSTOM FUNCTIONS
from functions import *

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/20 23:00:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [12]:
#### SPARK DATAFRAME
review = ps.read_json("./data/review.json", lines=True)
print(review.shape)

/opt/spark/python/pyspark/pandas/utils.py:975: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `read_json`, the default index is attached which can cause additional overhead.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


(6990280, 9)


In [13]:
#### TAKING A TINY SAMPLE FOR TRAINING A MODEL AND DROPPING DUPLICATES
sample = review.loc[:,['text', 'stars']].sample(frac=0.0005, random_state=1)
sample.drop_duplicates(inplace=True)
sample.drop_duplicates(subset='text', inplace=True)
sample.shape

(3552, 2)

In [161]:
sample.head()

,text,stars
3657455,"""We went there for brunch on my [mom's/sister'...",3
2919775,"""What's in Store"" is a costume jewelry and acc...",3
1000653,$2 per person for tea?! $8 for that (somewhat ...,4
5844865,$38 bucks that's what I was charged. $38 buck...,2
5644683,$9.99 for full service wash including car wash...,5


In [14]:
sample['stars'].unique()

0    1.0
1    4.0
2    3.0
3    2.0
4    5.0
Name: stars, dtype: float64

In [15]:
# Create a function to create a spacy dataset
def make_docs(data: List[Tuple[str, str]], target_file: str, cats: Set[str]):
    docs = DocBin()
    # Use nlp.pipe to efficiently process a large number of text inputs, 
    # the as_tuple arguments enables giving a list of tuples as input and 
    # reuse it in the loop, here for the labels
    for doc, label in nlp.pipe(data, as_tuples=True):
        # Encode the labels (assign 1 the subreddit)
        for cat in cats:
            doc.cats[cat] = 1 if cat == label else 0
        docs.add(doc)
    docs.to_disk(target_file)
    return docs

In [16]:
#### LABELS AS STRING AS NEEDED FOR SPACY
cats = sample['stars'].unique().astype(str).tolist()
cats

/opt/spark/python/pyspark/pandas/utils.py:975: PandasAPIOnSparkAdviceWarning: `to_list` loads all data into the driver's memory. It should only be used if the resulting list is expected to be small.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


['1.0', '4.0', '3.0', '2.0', '5.0']

In [17]:
#### TRANSFORMING TO PANDAS TO CLEAN THE DATA
sample = sample.to_pandas()

/opt/spark/python/pyspark/pandas/utils.py:975: PandasAPIOnSparkAdviceWarning: `to_pandas` loads all data into the driver's memory. It should only be used if the resulting pandas DataFrame is expected to be small.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


In [18]:
#### CLEANING THE DATA
sample = trim_all_columns(sample)
sample['text'] = normalize_column(sample, 'text')
pattern = '|'.join(['\n','\r', '\t' ,'\xa0','\u200b',','])
sample['text'] = clean_values(sample['text'], pattern, value=' ')
sample['stars'] = sample['stars'].astype(str)

In [19]:
sample['text'].head()

3657455    "We went there for brunch on my [mom's/sister'...
2919775    "What's in Store" is a costume jewelry and acc...
1000653    $2 per person for tea?! $8 for that (somewhat ...
5844865    $38 bucks  that's what I was charged. $38 buck...
5644683    $9.99 for full service wash including car wash...
Name: text, dtype: object

In [20]:
#### MAKING TEST TRAIN FOR SPACY MODELS
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(sample["text"].values,
                                    sample["stars"].values, test_size=0.3, 
                                    stratify=sample["stars"].values, random_state=1)

In [21]:
#### CREATING SPACY DATASET
make_docs(list(zip(X_train, y_train)), "train.spacy", cats=cats)
make_docs(list(zip(X_valid, y_valid)), "valid.spacy", cats=cats)

In [29]:
!pip list | grep thinc

thinc                             8.1.0


In [30]:
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 53.9 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [28]:
#### TRAINING THE MODEL WITH SPACY CLI
from spacy.cli.train import train as spacy_train

config_path = "./configs/config.cfg"
output_model_path = "./assets/model"
spacy_train(
    config_path,
    output_path=output_model_path,
    use_gpu = 0,
    overrides={
        "paths.train": "train.spacy",
        "paths.dev": "valid.spacy"
    }
)

ℹ Saving to output directory: assets/model
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['textcat_multilabel']
ℹ Initial learn rate: 0.001
E    #       LOSS TEXTC...  CATS_SCORE  SCORE 
---  ------  -------------  ----------  ------
  0       0           0.25       49.43    0.49
  0     200          31.71       57.03    0.57
  0     400          27.46       61.30    0.61
  0     600          26.11       61.91    0.62
  0     800          25.22       63.29    0.63
  0    1000          28.17       64.93    0.65
  0    1200          27.20       63.44    0.63
  0    1400          24.37       65.62    0.66
  1    1600          20.31       69.61    0.70
  1    1800          16.01       70.09    0.70
  1    2000          15.29       70.98    0.71
  1    2200          14.61       74.69    0.75
  2    2400           9.82       75.74  

In [147]:
#### TESTING THE MODEL
test = review.sample(frac=0.000005, random_state=3333)
test = test.to_pandas()
test = test[['text', 'stars']]

In [158]:
prueba1 = test['text'].sample().values[0]
prueba2 = test['text'].sample().values[0]

In [159]:
print(prueba1)
print(prueba2)

I was disappointed.  The place was dirty and the made no apologies for how dirty it was.  I kindly asked them to clean the condiments area and they refused.   I hope they make some changes soon. Also... The bathroom had flies.
Food was delicious. Drinks were delicious. Boudin balls were perfectly cooked and flavorful. I got the crab cake with poached egg (yum!). The same sauce is on it that the fried green tomato side was in so I was a little over the same taste after my meal but it was still good! I'll know for next time. Everyone else in our party had delicious brunch as well. Truffle Mac and cheese and cheese grits tasted similar so they just use the same cheese in both dishes.  The decor and ambiance of the place is very relaxing. Will be back!


In [160]:
#### LOADING THE MODEL AND TESTING
trained_nlp = spacy.load("./assets/model/model-last")
# Perform the trained pipeline on this text
doc1 = trained_nlp(prueba1)
doc2 = trained_nlp(prueba2)
# We can display the predicted categories
print(doc1.cats)
print(doc2.cats)

{'3': 0.04205531254410744, '4': 0.04049293324351311, '2': 0.6316655278205872, '5': 0.0632324069738388, '1': 0.5240751504898071}
{'3': 0.04304078221321106, '4': 0.005179326981306076, '2': 0.00015928283391986042, '5': 0.6787703633308411, '1': 1.00755369203398e-05}


Not tested yet things

In [ ]:
import cupy
import spacy
import multiprocessing as mp

from joblib import Parallel, delayed
from thinc.api import set_gpu_allocator, require_gpu

# A different method for flattening a list
def flatten2d(list2d):
    from functools import reduce
    from operator import iconcat
    return reduce(iconcat, list2d, [])

def chunker(iterator, length, chunksize):
    return (iterator[pos: pos + chunksize] for pos in range(0, length, chunksize))

def process_entity(doc):
    # I need lists of sentences for my use case,  but you could do other processing
    return [s.text for s in doc.sents]

def process_chunk(docs, rank):
    with cupy.cuda.Device(rank):
        set_gpu_allocator('pytorch')
        require_gpu(rank)
        nlp = spacy.load('en_core_web_sm', disable=['parser'])
        nlp.add_pipe('sentencizer')
        preprocess_pipe = []
        for doc in nlp.pipe(docs, batch_size=20):
            preprocess_pipe.append(process_entity(doc))
        rank += 1
        return preprocess_pipe

def process_parallel(docs, jobs=2, chunksize=50):
    executor = Parallel(n_jobs=jobs, prefer='threads')
    do = delayed(process_chunk)
    tasks = []
    gpus = list(range(0, cupy.cuda.runtime.getDeviceCount()))
    rank = 0
    for chunk in chunker(docs, len(docs), chunksize):
        tasks.append(do(chunk, rank))
        rank = (rank + 1) % len(gpus)
    result = executor(tasks)
    return flatten2d(result)

if __name__ == '__main__':
    preprocessed = process_parallel(
        docs = ["This is a basic sentence. This is another one."]*100,
        jobs=4,
        chunksize=25
    )
    print(preprocessed)

In [ ]:
from mpi4py import MPI
import cupy
from spacy.cli.train import train as spacy_train

config_path = "./configs/config.cfg"
output_model_path = "./assets/model"

comm = MPI.COMM_WORLD
rank = comm.Get_rank()

if rank == 0:
    data = ["His friend Nicolas J. Smith is here with Bart Simpon and Fred."*100]
else:
    data = None

unit = comm.scatter(data, root=0)

with cupy.cuda.Device(rank):
    import spacy
    from thinc.api import set_gpu_allocator, require_gpu
    set_gpu_allocator("pytorch")
    require_gpu(rank)
    nlp = spacy.load('en_core_web_lg')
    spacy_train(
    config_path,
    output_path=output_model_path,
    use_gpu = 0,
    overrides={
        "paths.train": "train.spacy",
        "paths.dev": "valid.spacy"
    }
)

    tmp_list = []
    for doc in nlp.pipe(unit):
        res = " ".join([t.text if not t.ent_type_ else t.ent_type_ for t in doc])
        tmp_list.append(res)

result = comm.gather(tmp_list, root=0)

if comm.rank == 0:
    print (result)
else:
    result = None